In [ ]:
import requests
import hashlib
import time
import datetime
from google.colab import userdata
import pandas as pd
import sqlite3

In [ ]:
public_key=userdata.get('public_key_marvel')
private_key=userdata.get('private_key_marvel')
timestamp = str(time.time())
md5_hash= timestamp + private_key + public_key
md5_hash = hashlib.md5(md5_hash.encode("utf-8")).hexdigest()

In [ ]:
# Modifique a URL para o endpoint de personagens
url = f"https://gateway.marvel.com/v1/public/characters?ts={timestamp}&apikey={public_key}&hash={md5_hash}"

# Código de tratamento dos dados
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    print(f"Total de personagens: {data['data']['total']}")
    for char in data['data']['results'][:5]:
        print(char)

In [ ]:
# Modifique a URL para o endpoint de quadrinhos
url = f"https://gateway.marvel.com/v1/public/comics?ts={timestamp}&apikey={public_key}&hash={md5_hash}"

# Código de tratamento dos dados
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    print(f"Total de quadrinhos: {data['data']['total']}")
    for comic in data['data']['results'][:5]:
        print(comic)

In [ ]:
data = pd.DataFrame(data['data']['results'])
data.head()

# Criação do banco de dados para os characters

In [ ]:
DB_NAME = 'marvel_full.db'
MAX_RETRIES = 3  # Tentativas por requisição
DELAY = 1  # Atraso entre requisições em segundos

In [ ]:
# Função para pegar todos os personagens
def get_all_characters():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    offset = 0
    total = None
    inserted = 0

    while True:
        for attempt in range(MAX_RETRIES):
            try:
                # Autenticação
                ts = str(time.time())
                hash_input = ts + private_key + public_key
                md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

                params = {
                    'ts': ts,
                    'apikey': public_key,
                    'hash': md5_hash,
                    'limit': 100,
                    'offset': offset
                }

                response = requests.get(
                    'https://gateway.marvel.com/v1/public/characters',
                    params=params,
                    timeout=10
                )
                response.raise_for_status()

                data = response.json()
                total = data['data']['total'] if total is None else total
                results = data['data']['results']

                # Inserir no banco
                for char in results:
                    cursor.execute('''
                        INSERT OR REPLACE INTO characters
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        char['id'],
                        char['name'],
                        char.get('description', ''),
                        char['modified'],
                        char['comics']['available'],
                        char['series']['available'],
                        char['stories']['available'],
                        char['events']['available'],
                        str(char['urls']),
                        f"{char['thumbnail']['path']}.{char['thumbnail']['extension']}"
                        if char['thumbnail'] else None
                    ))

                conn.commit()
                inserted += len(results)
                print(f"Progresso: {inserted}/{total} ({inserted/total:.1%})")

                # Verificar se terminou
                if offset + len(results) >= total:
                    print("\nTodos os personagens foram armazenados!")
                    return

                offset += 100
                time.sleep(DELAY)
                break  # Sai do loop de tentativas

            except requests.exceptions.RequestException as e:
                print(f"Erro na requisição (tentativa {attempt+1}): {str(e)}")
                time.sleep(2 ** attempt)  # Backoff exponencial
                continue
        else:
            print("Falha após múltiplas tentativas. Abortando...")
            break

    conn.close()

In [ ]:
# Função para criar o banco
def create_database():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS characters (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        description TEXT,
        modified TEXT,
        comics_available INTEGER,
        series_available INTEGER,
        stories_available INTEGER,
        events_available INTEGER,
        urls TEXT,
        thumbnail TEXT
    )''')

    conn.commit()
    conn.close()

In [ ]:
create_database()
get_all_characters()

# Verificar
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM characters")
print(f"\nTotal no banco: {cursor.fetchone()[0]} personagens")
conn.close()

# Mostrando o banco marvel_full

In [ ]:
def show_table_sample():
    conn = sqlite3.connect('marvel_full.db')
    df = pd.read_sql("SELECT * FROM characters", conn)
    conn.close()

    print("\nAmostra dos dados:")
    print(f"Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")
    return df

show_table_sample()